<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6f0bae2bab746f6c7a5cf071fc3195f132dbc0f351030d967152e338089bceea
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 09:14:04
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: 27.98 K (0.19%)
Current PnL: -19.93 L (-13.06%)
CY Booked + Current PnL: -6.41 L (-4.2%)
-------------------
Total profit:  2.01 L
Total loss:  -21.94 L
-------------------
Total Booked + Current PnL: 20.57 L (16.38%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.29%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 90.80 L (62.41%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.49%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,0.74,161051.0,-13508.0,13544.0,-0.00,-7.74,8.41,0.02,234.0,-1.00,1.11,40.09,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.40,86591.0,-14186.0,14279.0,-0.38,-14.08,16.49,0.09,253.0,-0.99,0.60,13.76,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.75,43.0,X-MC,3.59,88020.0,-2658.0,17569.0,0.00,-2.93,19.96,16.44,63.0,-0.15,0.61,16.65,X40,ATH,PHARMA
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200368.0,230.0,20658.0,0.39,0.11,10.31,10.44,5.0,0.01,1.38,5.68,X40,NTT,FMCG
35,ICICIGI,2252.93,-15.78,55.0,X-MC,7.46,187395.0,13075.0,22131.0,0.49,7.50,11.81,20.19,68.0,0.59,1.29,23.75,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,-282.78,45.0,H-SC,12.02,101575.0,-62030.0,183343.0,4.37,-37.91,180.50,74.15,149.0,-0.34,0.70,30.20,XR,NTT,CHEMICALS
56,RAJESHEXPO,518.0,1783.84,67.0,L-SC,2.76,51410.0,-85767.0,85860.0,3.56,-62.52,167.01,0.07,268.0,-1.00,0.35,27.86,OX40N,NTT,JEWELLERY
0,5PAISA,593.0,79.09,24.0,H-SC,17.09,121888.0,-96608.0,124801.0,1.33,-44.21,102.39,12.90,161.0,-0.77,0.84,1.35,OX40N,NTT,FINANCE
69,STARHEALTH,761.0,39.64,74.0,H-SC,15.60,287477.0,6354.0,126519.0,1.15,2.26,44.01,47.26,171.0,0.05,1.98,54.02,XY24,NTT,INSURANCE
64,SFL,1287.0,29.90,44.0,M-SC,2.88,171493.0,-90725.0,160552.0,1.09,-34.60,93.62,26.63,239.0,-0.57,1.18,6.62,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-19.41,29.0,X-LC,0.36,248900.0,-27510.0,448667.0,-2.86,-9.95,180.26,152.37,3.0,-0.06,1.71,0.00,XY24,NTT,AUTO
19,COFFEEDAY,80.00,-51.71,34.0,L-SC,21.21,72307.0,-41242.0,81295.0,-1.93,-36.32,112.43,35.27,270.0,-0.51,0.50,76.97,XR,NTT,HOTELS
54,PGHH,17907.65,-31.71,35.0,X-MC,2.09,194265.0,-6555.0,74345.0,-0.65,-3.26,38.27,33.76,57.0,-0.09,1.34,1.76,X40,ATH,FMCG
24,EASEMYTRIP,26.40,-20.01,41.0,M-SC,20.17,77088.0,-99727.0,180525.0,-0.63,-56.40,234.18,45.70,205.0,-0.55,0.53,0.64,XY24,NTT,TRAVEL
71,SYMPHONY,1306.00,-45.70,38.0,M-SC,15.32,115804.0,-55337.0,55285.0,-0.47,-32.33,47.74,-0.03,206.0,-1.00,0.80,1.07,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,8.91,114540.0,-258.0,94301.0,0.00,-0.22,82.33,81.92,236.0,-0.0,0.79,35.60,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.52,48.0,M-SC,10.22,147730.0,86.0,159238.0,0.89,0.06,107.79,107.91,216.0,0.0,1.02,3.06,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.08,39.0,H-MC,5.61,209508.0,-66.0,74732.0,0.38,-0.03,35.67,35.63,119.0,-0.0,1.44,16.64,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151261.0,316.0,109135.0,-0.16,0.21,72.15,72.51,99.0,0.0,1.04,6.24,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,48.0,M-SC,10.22,147730.0,86.0,159238.0,0.89,0.06,107.79,107.91,216.0,0.00,1.02,3.06,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,0.74,161051.0,-13508.0,13544.0,-0.00,-7.74,8.41,0.02,234.0,-1.00,1.11,40.09,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.40,86591.0,-14186.0,14279.0,-0.38,-14.08,16.49,0.09,253.0,-0.99,0.60,13.76,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.79,53.0,H-SC,2.41,225000.0,-44667.0,81000.0,0.48,-16.56,36.00,13.47,158.0,-0.55,1.55,14.41,XY24,NTT,PAINTS
67,SIS,528.00,2023.32,47.0,H-SC,2.36,85680.0,-25352.0,48958.0,-0.06,-22.83,57.14,21.26,166.0,-0.52,0.59,15.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,39.64,74.0,H-SC,15.60,287477.0,6354.0,126519.0,1.15,2.26,44.01,47.26,171.0,0.05,1.98,54.02,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,17.65,53.0,M-SC,36.83,159300.0,10080.0,134099.0,0.49,6.76,84.18,96.62,245.0,0.08,1.10,72.40,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151261.0,316.0,109135.0,-0.16,0.21,72.15,72.51,99.0,0.00,1.04,6.24,XR,NTT,IT
25,FINCABLES,1641.55,-18.52,48.0,M-SC,10.22,147730.0,86.0,159238.0,0.89,0.06,107.79,107.91,216.0,0.00,1.02,3.06,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.43,130576.0,7240.0,124387.0,0.50,5.87,95.26,106.72,139.0,0.06,0.90,30.62,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.43,130576.0,7240.0,124387.0,0.50,5.87,95.26,106.72,139.0,0.06,0.90,30.62,AR,ATH,MISC
25,FINCABLES,1641.55,-18.52,48.0,M-SC,10.22,147730.0,86.0,159238.0,0.89,0.06,107.79,107.91,216.0,0.00,1.02,3.06,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,8.91,114540.0,-258.0,94301.0,0.00,-0.22,82.33,81.92,236.0,-0.00,0.79,35.60,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151261.0,316.0,109135.0,-0.16,0.21,72.15,72.51,99.0,0.00,1.04,6.24,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.08,39.0,H-MC,5.61,209508.0,-66.0,74732.0,0.38,-0.03,35.67,35.63,119.0,-0.00,1.44,16.64,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.0,11.77,26.0,X-SC,9.73,85249.0,-43421.0,88719.0,-0.00,-33.75,104.07,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.0,-53.17,28.0,X-SC,37.26,46461.0,-18545.0,169476.0,0.15,-28.53,364.77,232.18,83.0,-0.11,0.32,0.15,XY24,NTT,MISC
76,TMPV,1065.0,-19.41,29.0,X-LC,0.36,248900.0,-27510.0,448667.0,-2.86,-9.95,180.26,152.37,3.0,-0.06,1.71,0.00,XY24,NTT,AUTO
27,GLAXO,3466.2,-19.65,30.0,X-MC,6.40,199336.0,52.0,77960.0,0.50,0.03,39.11,39.15,60.0,0.00,1.37,29.04,X40,ATH,PHARMA
46,JSWINFRA,342.0,-19.06,32.0,X-MC,5.67,189675.0,-10672.0,41178.0,0.04,-5.33,21.71,15.23,66.0,-0.26,1.30,25.21,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-19.41,29.0,X-LC,0.36,248900.0,-27510.0,448667.0,-2.86,-9.95,180.26,152.37,3.0,-0.06,1.71,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200368.0,230.0,20658.0,0.39,0.11,10.31,10.44,5.0,0.01,1.38,5.68,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,36.0,X-SC,1.00,106455.0,-21483.0,68621.0,-0.13,-16.79,64.46,36.84,90.0,-0.31,0.73,8.98,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.34,65.0,X-MC,1.17,184004.0,7266.0,47473.0,-0.23,4.11,25.80,30.97,75.0,0.15,1.26,17.19,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-4.98,48.0,H-LC,1.64,154670.0,-31425.0,78913.0,0.30,-16.89,51.02,25.51,49.0,-0.40,1.06,13.98,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.77,26.0,X-SC,9.73,85249.0,-43421.0,88719.0,-0.00,-33.75,104.07,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-19.41,29.0,X-LC,0.36,248900.0,-27510.0,448667.0,-2.86,-9.95,180.26,152.37,3.0,-0.06,1.71,0.00,XY24,NTT,AUTO
55,QUESS,986.00,-53.17,28.0,X-SC,37.26,46461.0,-18545.0,169476.0,0.15,-28.53,364.77,232.18,83.0,-0.11,0.32,0.15,XY24,NTT,MISC
53,PAGEIND,51769.92,-28.09,41.0,X-MC,7.72,159980.0,-3760.0,47098.0,0.58,-2.30,29.44,26.47,55.0,-0.08,1.10,1.36,X40,ATH,APPARELS
54,PGHH,17907.65,-31.71,35.0,X-MC,2.09,194265.0,-6555.0,74345.0,-0.65,-3.26,38.27,33.76,57.0,-0.09,1.34,1.76,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,28.0,X-SC,37.26,46461.0,-18545.0,169476.0,0.15,-28.53,364.77,232.18,83.0,-0.11,0.32,0.15,XY24,NTT,MISC
59,RELAXO,1176.00,-41.05,41.0,X-SC,3.23,80379.0,-64781.0,140711.0,0.33,-44.63,175.06,52.31,91.0,-0.46,0.55,7.59,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,11.77,26.0,X-SC,9.73,85249.0,-43421.0,88719.0,-0.00,-33.75,104.07,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.75,43.0,X-MC,3.59,88020.0,-2658.0,17569.0,0.00,-2.93,19.96,16.44,63.0,-0.15,0.61,16.65,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,36.0,X-SC,1.00,106455.0,-21483.0,68621.0,-0.13,-16.79,64.46,36.84,90.0,-0.31,0.73,8.98,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,60.0,X-LC,8.48,291964.0,-53992.0,120698.0,0.00,-15.61,41.34,19.28,1.0,-0.45,2.01,7.93,X40,ATH,IT
41,INFY,2275.00,-20.67,51.0,X-LC,4.01,318648.0,5760.0,165920.0,-0.45,1.84,52.07,54.87,2.0,0.03,2.19,10.28,X40,BTT,IT
76,TMPV,1065.00,-19.41,29.0,X-LC,0.36,248900.0,-27510.0,448667.0,-2.86,-9.95,180.26,152.37,3.0,-0.06,1.71,0.00,XY24,NTT,AUTO
82,VBL,671.64,-17.64,49.0,X-LC,5.98,295394.0,-20448.0,133105.0,0.78,-6.47,45.06,35.67,4.0,-0.15,2.03,6.48,X40N,ATH,FMCG
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200368.0,230.0,20658.0,0.39,0.11,10.31,10.44,5.0,0.01,1.38,5.68,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,34.0,L-SC,21.21,72307.0,-41242.0,81295.0,-1.93,-36.32,112.43,35.27,270.0,-0.51,0.50,76.97,XR,NTT,HOTELS
7,ASIANTILES,137.00,7638.89,65.0,L-SC,17.15,84891.0,-8919.0,85536.0,0.12,-9.51,100.76,81.67,271.0,-0.10,0.58,64.39,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.25,52.0,H-SC,7.48,92775.0,-5205.0,26812.0,0.00,-5.31,28.90,22.05,164.0,-0.19,0.64,34.86,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.43,130576.0,7240.0,124387.0,0.50,5.87,95.26,106.72,139.0,0.06,0.90,30.62,AR,ATH,MISC
70,SURYODAY,240.00,53.51,43.0,H-SC,13.29,140842.0,-38229.0,103955.0,0.40,-21.35,73.81,36.71,167.0,-0.37,0.97,39.74,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.63,86.0,X-LC,9.54,255763.0,3995.0,49209.0,-0.00,1.59,19.24,21.13,31.0,0.08,1.76,37.57,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,0.74,161051.0,-13508.0,13544.0,-0.00,-7.74,8.41,0.02,234.0,-1.00,1.11,40.09,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.45,149686.0,-33089.0,144163.0,0.11,-18.10,96.31,60.77,159.0,-0.23,1.03,34.80,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7638.89,65.0,L-SC,17.15,84891.0,-8919.0,85536.0,0.12,-9.51,100.76,81.67,271.0,-0.10,0.58,64.39,XR,NTT,CERAMICS
5,ANGELONE,3033.00,60.95,75.0,X-SC,7.33,226238.0,35232.0,22465.0,0.50,18.45,9.93,30.21,97.0,1.57,1.56,43.18,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,44.52
2,50,75.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.65
MC    27.73
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.48
X40      20.50
X40N     12.28
XY25     10.05
XR        9.56
AR        9.00
OX40N     7.56
X200      1.80
SR        1.00
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.28
X-MC    21.37
X-LC    19.79
M-SC    13.37
X-SC     8.57
H-MC     4.67
H-LC     2.86
L-SC     1.43
M-MC     1.36
M-LC     1.04
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.68,-3.78,37.33
IT,12.46,-16.06,78.10
FINANCE,11.95,-8.92,58.30
MISC,7.55,-18.35,78.05
PAINTS,6.03,-5.34,21.04
ELECTRICAL,5.55,-11.88,51.55
INSURANCE,4.53,4.05,29.77
PHARMA,3.42,-0.54,34.27
AUTO,3.38,-16.56,128.30


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3310525.0,20
AR,1263993.0,10
XR,1263835.0,13
X40,989115.0,14
OX40N,694291.0,10
X40N,683741.0,9
XY25,473134.0,7
SR,274401.0,2
MH,65473.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3474326.0,25
M-SC,1409066.0,16
X-LC,1191028.0,11
X-MC,1134551.0,15
X-SC,830522.0,8
H-MC,400613.0,3
L-SC,252691.0,3
H-LC,140838.0,2
M-LC,109135.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1164886.0      6
           AR         842052.0      5
           XR         791122.0      7
M-SC       XY24       734911.0      6
X-LC       XY24       509731.0      2
X-MC       X40        412380.0      7
X-LC       X40        407988.0      5
X-SC       XY24       375734.0      2
X-MC       XY24       342371.0      3
H-SC       OX40N      336392.0      4
X-SC       X40N       286041.0      4
H-SC       SR         274401.0      2
M-SC       OX40N      272039.0      5
H-MC       AR         217721.0      2
X-LC       X40N       199687.0      2
X-MC       X40N       198013.0      3
H-MC       XY24       182892.0      1
X-MC       XY25       181787.0      2
X-SC       X40        168747.0      2
L-SC       XR         166831.0      2
M-SC       XR         142710.0      2
           XY25       134099.0      1
           AR         125307.0      2
M-LC       XR         109135.0      1
L-SC       OX40N       85860.0      1
H-LC       AR          78913.0      1
X-LC       XY25        73622.0      2
H-SC       MH          65473.0      1
H-LC       X200        61925.0      1
L-MC       XR          54037.0      1
M-MC       XY25        47428.0      1
L-LC       XY25        36198.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
